In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bz2
import json
from glob import glob
from uuid import uuid4
from os.path import isdir
from tqdm.notebook import tqdm
from elasticsearch import Elasticsearch

In [3]:
elastic = Elasticsearch(
    hosts='https://corpus.ncsoft.com:9200',
    http_auth='elastic:nlplab',
    verify_certs=False,
    ssl_show_warn=False,
    http_compress=True,
)

In [4]:
def import_wiki(lang):
    """ """
    index = 'corpus-wiki-{}'.format(lang)
    path = '../data/wiki/{}wik*/*/*.bz2'.format(lang)

    for f in tqdm(glob(path), desc=lang):
        with bz2.open(f, 'rb') as fp:
            buf = []
            for l in tqdm(fp.readlines(), desc=f):
                doc = json.loads(l.decode('utf-8'))

                buf += [{
                    'update': {
                        '_id': str(uuid4()),
                        '_index': index,
                    }
                }, {
                    'doc': doc,
                    'doc_as_upsert': True,
                }]

                if len(buf) > 2000:
                    elastic.bulk(
                        index=index,
                        body=buf,
                        refresh=True,
                        params={'request_timeout': 620},
                    )
                    buf = []

            if len(buf) > 1:
                elastic.bulk(
                    index=index,
                    body=buf,
                    refresh=True,
                    params={'request_timeout': 620},
                )

In [7]:
# ko,zh,en,ja,vi,id,th
for lang in 'ja,vi,id,th'.split(','):
    import_wiki(lang=lang)